## INFORMAÇÕES DO ALUNO

In [0]:
'''
SOFTWARES PARA USO EM CIÊNCIA DE DADOS II
PROJETO 02/02 - Beer Consumption
PROFESSOR: NASSER BOAN
ALUNO: ARTHUR FARIAS
MATRÍCULA: 1931133115
'''

# RESUMO E RESULTADOS

In [0]:
'''

Este projeto objetiva predizer o consumo de cerveja pra cidade de São Paulo com base em algumas informações 
disponibilizadas.
Após avaliação inicial do nosso arquivo de dados, verificamos que as configurações regionais não seguiam os padrões. 
Assim, a importação dos dados foi realizada definindo que todas as variáveis seriam do tipo "string" e que uma 
posterior adequação seria feita. Também foi realizada uma renomeação das variáveis para facilitar o trabalho.

Iniciando o processo de análise exploratória dos dados, foram identificados muitos dados "missing". Com isso, optamos
por eliminar as linhas onde todas as variáveis tivessem valores nulos. Em seguida iniciando o tratamento das colunas, 
adequando-as para que pudessem ser transformadas em "data types" de tipos mais adequados para modelos de Regressão linear.
Realizemos também a criação de novas "features" que posteriormente não se mostraram promissoras na explicação que desejávamos.
Por fim, dentro deste subtópico, apesar do box-plot indicar possíveis outliers para algumas variáveis, os testes com 
filtros não melhoraram nosso resultado.

Uma vez definidas as variáveis que comporiam o modelo, iniciamos as regressões:

Na Regressão Linear Simples a variável temperatura máxima foi a que melhor explicou o consumo de cerveja obtendo precisão
pouco superior a 41%.
Na Regressão Linear Múltipla o resultado foi de 65,6% de acerto na partição de teste e 70,7% no treino. O que já era um
resultado melhor do que na Regressão Linear Simples.
Por fim, na Regressão Polinomial o grau 2 foi o que apresentou uma leve melhora, elevando o resultado de teste e treino para
65,8% e 70,8% respectivamente, sendo o melhor resultado do projeto.

Assim, nosso modelo apesar de não acertar numa faixa excelente, apresenta-se sem vícios e com bom nível de precisão.

'''

# IMPORTANDO DATASET E BIBLIOTECAS

## Bibliotecas

In [1]:
## IMPORTANDO AS BIBLIOTECAS QUE SERÃO UTILIZADAS NO PROJETO ##
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score,train_test_split

# Comment this if the data visualisations doesn't work on your side
# %matplotlib inline

# We are using whitegrid style for our seaborn plots. This is like the most basic one
sns.set_style(style = 'whitegrid')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# SE O GOOGLE DRIVE NÃO QUISER MONTAR
from google.colab import drive
drive.mount('/content/drive')
#4/zQErY0XYvMT8e9M6WUlfE75iFXPdCAL-FdCOd2ecZLxtibMqDqsN39c

## Dataset

In [0]:
## IMPORTANDO A BASE E REALIZANDO OS TRATAMENTOS INICIAIS
df = pd.read_csv('/content/drive/My Drive/git/Beer_Consumption/data/Consumo_cerveja.csv',sep=',',encoding='iso-8859-1', dtype=str)
df.columns = [x.lower() for x in df.columns]
df_columns = df.columns
df.head(3)

In [0]:
# MELHORANDO O NOME DAS VARIÁVEIS
df.columns = ['data', 't_media', 't_minima', 't_maxima', 'precipitacao', 'fim_semana', 'consumo']
df_columns = df.columns
df.head(3)

In [0]:
# DICIONÁRIO DE DADOS

'''
data            Data
t_media         Temperatura Media (C)
t_minima        Temperatura Minima (C)
t_maxima        Temperatura Maxima (C)
precipitacao    Precipitacao (mm)
fim_semana      Final de Semana
consumo         Consumo de cerveja (litros)
'''

# ANÁLISE EXPLORATÓRIA

In [0]:
# VERIFICANDO O SHAPE DO DATASET E CRIANDO VARIÁVEIS
n_rows    = df.shape[0]
n_cols    = df.shape[1]
v_cols    = df.columns
v_answer  = 'consumo'

print('O Dataset possui' , n_rows, 'linhas e' , n_cols , 'colunas.')

In [0]:
# IDENTIFICANDO VALORES NULOS
if df.isnull().values.any() == True:
  print(df.isna().sum(), '\n\n\nVALORES PERCENTUAIS DE NULO\n')
  print((df.isnull().sum()/n_rows).round(2)*100)
else:
  print('O Dataset não possui valores nulos.')

In [0]:
# REMOVENDO AS LINHAS ONDE TODAS AS VARIÁVEIS TIVEREM VALORES NULO
df = df.dropna(thresh=7)

In [0]:
# IDENTIFICANDO VALORES NULOS
if df.isnull().values.any() == True:
  print(df.isna().sum(), '\n\n\nVALORES PERCENTUAIS DE NULO\n')
  print((df.isnull().sum()/n_rows).round(2)*100)
else:
  print('O Dataset não possui valores nulos.')

In [0]:
# VERIFICANDO INFORMAÇÕES DO DATASET
df.info(verbose=True)

In [0]:
# AJUSTANDO AS INFORMAÇÕES PARA O PADRÃO IMPERIAL
# MODIFICANDO OS TIPOS DE VARIÁVEIS
df['consumo']       = [x.replace('.', '') for x in df['consumo']]
df['t_media']       = [x.replace(',', '.') for x in df['t_media']]
df['t_minima']      = [x.replace(',', '.') for x in df['t_minima']]
df['t_maxima']      = [x.replace(',', '.') for x in df['t_maxima']]
df['precipitacao']  = [x.replace(',', '.') for x in df['precipitacao']]

df['data']          = pd.to_datetime(df.data, format='%Y-%m-%d')
df['t_media']       = df['t_media'].astype('float64')
df['t_minima']      = df['t_minima'].astype('float64')
df['t_maxima']      = df['t_maxima'].astype('float64')
df['precipitacao']  = df['precipitacao'].astype('float64')
df['fim_semana']    = df['fim_semana'].astype('int64')
df['consumo']       = df['consumo'].astype('int64')

In [0]:
# VERIFICANDO INFORMAÇÕES DO DATASET
df.info(verbose=True)

In [0]:
# NÚMERO DE VALORES DISTINTOS NAS VARIÁVEIS
print(df.nunique(),'\n\nRELAÇÃO DE VARIÁVEIS COM ATÉ 20 VALORES DISTINTOS')

for i in v_cols:
  if df[i].nunique() <= 20:
    print(i,sorted(df[i].unique()))
#    print(df[i].value_counts(dropna=False))

In [0]:
# CRIAÇÃO DE NOVAS FEATURES
df['data_dia']        = df['data'].dt.day
df['data_mes']        = df['data'].dt.month
df['data_nrsemana']   = df['data'].dt.week
df['data_diasemana']  = df['data'].dt.dayofweek
df['choveu']          = np.where(df['precipitacao'] > 0 , 1, 0)
df['chuva_fraca']     = np.where(((df['precipitacao'] > 0) & (df['precipitacao'] < 10)) , 1, 0)
df['chuva_moderada']  = np.where(((df['precipitacao'] >= 10) & (df['precipitacao'] < 20)) , 1, 0)
df['chuva_forte']     = np.where(((df['precipitacao'] >= 20) & (df['precipitacao'] < 30)) , 1, 0)
df['temporal']        = np.where(df['precipitacao'] >= 30 , 1, 0)

In [0]:
# ESTATISTICA DESCRITIVA DOS DADOS
df.describe().round(2)

In [0]:
# ALTERANDO A VISÃO PARA MELHORAR A VISUALIZAÇÃO
df.describe().round(2).T

In [0]:
# GRÁFICO COM A DISTRIBUIÇÃO DOS VALORES
gr_dist = df.hist(figsize = (20,10))

In [0]:
# BOX-PLOT DAS VARIÁVEIS EM BUSCA DE OUTLIERS
ix = 1
fig = plt.figure(figsize = (15,10))
for c in list(v_cols_num):
    if ix <= 3:
      ax1 = fig.add_subplot(2,3,ix)
      sns.boxplot(y=df[c], ax=ax1)
      ax1.set_title('Box plot - {}'.format(c), fontsize=10)
      ax1.set_xlabel(c, fontsize=8)
    ix = ix +1
    if ix == 4: 
        fig = plt.figure(figsize = (15,10))
        ix =1

In [0]:
# AVALIAÇÃO DOS LIMITES
for i in v_cols_num:
  factor = 3
  iq = np.quantile(df[i], .75) - np.quantile(df[i], .25)
  upper_lim = np.quantile(df[i], .75) + iq * factor
  lower_lim = np.quantile(df[i], .25) - iq * factor
  v_cont_u = (df[i][(df[i] < upper_lim)].count()) - n_rows
  v_cont_l = (df[i][(df[i] > lower_lim)].count()) - n_rows
  print('Se aplicado a regra de UPPER INNER FENCE a variável',i,'eliminará',v_cont_u,'linhas.',iq,upper_lim,lower_lim)
  print('Se aplicado a regra de LOWER INNER FENCE a variável',i,'eliminará',v_cont_l,'linhas.')

In [0]:
# VERIFICANDO A CORRELAÇÃO ENTRE AS VARIÁVEIS
plt.figure(figsize=(20,5))
sns.heatmap(df.corr(),annot=True)

In [0]:
# PELA MATRIZ DE CORRELAÇÃO ELIMINAMOS AS VARIÁVIES COM CORRELAÇÃO MUITO FRACA, INFERIOR A .20
df_tmp = df[['t_media','t_minima','t_maxima','fim_semana','data_diasemana','consumo']]

In [0]:
# VERIFICANDO A CORRELAÇÃO ENTRE AS VARIÁVEIS
plt.figure(figsize=(20,5))
sns.heatmap(df_tmp.corr(),annot=True)

In [0]:
# ELIMINAMOS AS VARIAVEIS COM ALTA CORRELAÇÃO ENTRE SI, SUPERIOR A .80
df1 = df_tmp[['t_maxima','t_minima','fim_semana','consumo']]

In [0]:
# CRIANDO VARIAVEIS POR CATEGORIA DA COLUNA
v_cols_num = []
v_cols_other = []
for i in df1.columns:
    if(df1[i].dtype == "float64" or df1[i].dtype == "int64"):
          v_cols_num.append(i)
    else:
          v_cols_other.append(i)
print('\t',v_cols_num,'\n\t',v_cols_other)

In [0]:
# VERIFICANDO A RELAÇÃO DAS VARIÁVEIS INDEPENDENTES COM A VARIÁVEL DEPENDENTE
ix = 1
fig = plt.figure(figsize = (15,10))
for c in list(v_cols_num):
    if ix <= 3:
      if c != v_answer:
        ax1 = fig.add_subplot(2,3,ix)
        sns.scatterplot(x=df1[c],y=df1[v_answer],s=100)
    ix = ix +1
    if ix == 4: 
        fig = plt.figure(figsize = (15,10))
        ix =1

# REGRESSÃO LINEAR SIMPLES

In [0]:
# REGRESSÃO LINEAR SIMPLES
lr_s  = LinearRegression()
y     = df1[v_answer]

for a,i in enumerate(v_cols_num):
  if i != v_answer:
    v_x = df1[i].values.reshape(-1,1)
    lr_s.fit(v_x,y)
    v_erro = mean_squared_error(y,lr_s.predict(v_x))
    r2 = r2_score(y,lr_s.predict(v_x)).round(3)
    v2_erro = round(np.sqrt(v_erro),2)
    print(i.upper(),'\nMeu erro foi de\t\t',round(v_erro,2),'\nErro na mesma escala\t',v2_erro,'\nA precisão foi de\t',r2,'\n\t\t\t',lr_s.coef_,lr_s.intercept_,'\n')
    if a == 0:
      vs_erro = v2_erro
      vs_param = i
      vs_r = r2
    else:
      if v2_erro < vs_erro:
        vs_erro = v2_erro
        vs_param = i
        vs_r = r2

print('Meu melhor parametro na regressão linear simples foi',vs_param,'com um erro de',vs_erro,'e precisão de',vs_r)

In [0]:
# REGRESSÃO LINEAR SIMPLES - GRÁFICO
for i in v_cols_num:
  if i != v_answer:
    v_x = df1[i].values.reshape(-1,1)
    lr_s.fit(v_x,y)
    
    y_pred= [lr_s.predict([[x]])[0] for x in df1[i]]
    plt.figure(figsize=(8,4))
    sns.scatterplot(x=df1[i],y=y,s=100)
    sns.lineplot(x=df1[i],y=y_pred,color='tab:orange')
    plt.show

# REGRESSÃO LINEAR MÚLTIPLA

In [0]:
# CRIANDO DATASET'S DE TREINO E TESTE - TIMESTAMP NÃO É UMA COLUNA ACEITA NA REGRESSÃO
x = df1.drop([v_answer], axis=1)
y = df1[v_answer]
#x = df.drop(['data','t_media','data_mes','precipitacao','data_dia','t_minima', v_answer], axis=1)
#x = x[x['t_maxima'] > 15.4]
#v_answer = df.consumo[df['t_maxima'] > 15.4]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=28)

In [0]:
# RELAÇÃO LINEAR MULTIPLA - PESOS DAS MINHAS VARIÁVEIS
lr_m = LinearRegression()
lr_m.fit(x_train,y_train)
for (i_lr,col) in zip(lr_m.coef_.round(2),x.columns):
  print('O Coeficiente da variável',col,'\tfoi ',i_lr)

In [0]:
# GRÁFICO COM OS PESOS DA MINHA REGRESSÃO
plt.figure(figsize=(25,8))
sns.barplot(x = x.columns, y = lr_m.coef_)
plt.show()

In [0]:
# CALCULANDO O MEU ERRO PARA REGRESSÃO LINEAR MULTIPLA
v_erro = mean_squared_error(y_test,lr_m.predict((x_test)))
vm_erro = np.sqrt(v_erro).round(2)
if vm_erro < vs_erro:
  print('Meu erro em uma regressão multipla foi de',vm_erro,', melhor resultado do que em uma regressão linear simples',vs_erro)
else:
  print('Meu erro em uma regressão simples foi de',vs_erro,', melhor resultado do que em uma regressão linear multipla',vm_erro)

In [0]:
# VERIFICAMOS COM O TESTE ANTERIOR QUE NA REGRESSÃO LINEAR MULTIPLA QUE MEU MODELO, COMO ERA DE SE ESPERAR, JÁ ERRA MENOS DO QUE COM APENAS UMA VARIÁVEL.

In [0]:
# CALCULANDO A PRECISÃO DO MEU MODELO
prm_treino = lr_m.score(x_train,y_train)
prm_teste = lr_m.score(x_test,y_test)

print('Meu percentual de acerto no treino foi de',prm_treino.round(3),'\nMeu percentual de acerto no teste foi de',prm_teste.round(3))
#y_pred= lr.predict(x_test)
#r2_score(y_test,y_pred)

# r2 < 60% é ruim
# r2 > 80% é muito bom
# r2 = 1 é ruim

In [0]:
r2_score(y_test,lr_m.predict(x_test))

# REGRESSÃO POLINOMIAL

In [0]:
# TESTANDO MEU ERRO DE TREINO/TESTE NUM LOOP - IGUAL QUE NEM O DO PROFESSOR
x = df1.drop([v_answer], axis=1)
y = df1[v_answer]

trn_error = []
tst_error = []
prm_tst = []
prm_trn = []

lr_p = LinearRegression()

for i in range(1,8):
  poly = PolynomialFeatures(degree=i)
  x_poly = poly.fit_transform(x)

  xp_train, xp_test, yp_train, yp_test = train_test_split(x_poly, y, test_size=0.25,random_state=28)

  lr_p.fit(xp_train,yp_train)

  ytst_pred = lr_p.predict(xp_test)
  ytrn_pred = lr_p.predict(xp_train)

  tst_error.append(mean_squared_error(yp_test, ytst_pred))
  trn_error.append(mean_squared_error(yp_train, ytrn_pred))
  prm_tst.append(lr_p.score(xp_test, yp_test))
  prm_trn.append(lr_p.score(xp_train, yp_train))


In [0]:
# GRÁFICO DOS ERROS GERADOS PELO LOOP
plt.figure(figsize=(10,7))

trn_error_v = np.log10(trn_error)
tst_error_v = np.log10(tst_error)

sns.lineplot(x = list(range(1,8)), y = trn_error_v, label='trn_error',color='b',marker='D')
sns.lineplot(x = list(range(1,8)), y = tst_error_v, label='tst_error',color='r',marker='X')

plt.title('Erro de Treino e Teste', size = 15)

plt.legend()
plt.show()

In [0]:
# A PRECISÃO DOS MEUS MODELOS POLINOMIAIS FOI
i = 0
for (a,b) in zip(prm_tst,prm_trn):
  i += 1
  print('Com polinomio grau',i,'\n\tPrecisão no teste ',a.round(3),'\n\tPrecisao no treino',b.round(3))

In [0]:
'''
Nesse projeto foi 
'''